In [ ]:
# ANALYSIS TASK 3
#  Is there any correlation in the time series data to see if the crypto trends reflect the stock trends in price and volume changes?

In [59]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
data_main = sqlContext.read.csv('data.csv',header=False)

In [60]:
# Adding an index column
dates = data_main.rdd.map(lambda x: x[0]).collect()
# Order of columns
# 'INDEX','DATE', 'C_PCP', 'C_TMVCP', 'N_PCP', 'N_TMVCP', 'SP_PCP', 'SP_TMVCP', 'NT_PCP', 'NT_TMVCP'
data_main = data_main.rdd.map(lambda x: (dates.index(x[0]),x[0],float(x[1]),float(x[2]),float(x[3]),float(x[4]),float(x[5]),float(x[6]),float(x[7]),float(x[8])) )
data_main.toDF().show(5)

+---+----------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+-------------------+-------------------+
| _1|        _2|                 _3|                _4|                 _5|                _6|                  _7|                 _8|                 _9|                _10|
+---+----------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+-------------------+-------------------+
|  0|02-07-2018|  5.121125072620959| 9.193665598572519|-1.4197329157191518|               0.0|   -6.92142945367664|                0.0|-1.3828243249559322|                0.0|
|  1|03-07-2018|-1.7263876931070241|  3.88579704445608|-0.7503701677520215|-34.69465041770667|-0.38754218086889713|-31.618496889332604|-1.4233110883018818| -32.83890509077797|
|  2|05-07-2018|-0.7984669906309387| 5.925269446030441| 1.2241598859767095| 70.59531435527903|  0.9503111182226929|  58.

In [61]:
# before covid
data = data_main.map(lambda x: (x[0],x[1],x[4],x[2]))
# Before covid
from datetime import datetime
def reformat_date(date_time_str):
    dateitem = datetime.strptime(date_time_str, '%d-%m-%Y')
    return dateitem
date = datetime.strptime('31-12-2019', '%d-%m-%Y')
data = data.filter(lambda x: reformat_date(x[1])<date)
print(data.count())
data_df = data.toDF(('INDEX','DATE','N_PCP','C_PCP'))
data_df.registerTempTable('TABLE')

377


In [52]:
# cross correlation - time lagged
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# price change percent
# crypto and nasdaq100
# we are trying to study the effect of N_PCP change trends on C_PCP change trends
# creating time lagged values for C_PCP for upto 30 days
for i in range(1,31):
    window = Window.orderBy("Index")
    leadCol = F.lead("C_PCP", i).over(window)
    data_dfx = data_df.withColumn("C_PCP+"+str(i), leadCol)
    data_dfx.registerTempTable('DATA')